# Tracking DSPy Optimizers with MLflow

[Tracking DSPy Optimizers with MLflow](https://dspy.ai/tutorials/optimizer_tracking/)

This tutorial demonstrates how to use [MLflow](https://mlflow.org/) to track and analyze your DSPy optimization process.

MLflow's built-in integration for [DSPy](https://dspy.ai/) (known as [MLflow DSPy Flavor](https://mlflow.org/docs/latest/genai/flavors/dspy/)) gives you the following features:

1. Provides traceability and debuggability for your DSPy optimization experience.
1. Allows you to understand the intermediate trials during the optimization, store the optimized program and its results, and provides observability into your program execution.


## 📚 Articles

1. https://medium.com/@AI-on-Databricks/boosting-dspy-optimizer-observability-with-mlflow-698b6dc29ea7
1. [DSPy + MLflow for Automatically Optimizing LLM Programs](https://notebooks.databricks.com/devrel/mlflow/2024-11-27-dspy.html)
1. [MLflow Pipeline with H2O AutoML and DSPy](https://gist.github.com/ruvnet/772f5b12d3df4027f5c7952186cb0d1c)
1. [Tutorial: Debugging and Observability in DSPy](https://dspy.ai/tutorials/observability/)
1. (video) [Streamlining DSPy Development: Track, Debug, and Deploy With MLflow](https://www.youtube.com/watch?v=LlGuO_MyTXU)

In [0]:
%pip install -qU dspy>=3.0.4 mlflow>=3.7.0
dbutils.library.restartPython()

In [0]:
import dspy

print(f"DSPy version: {dspy.__version__}")

In [0]:
import mlflow

print(f"MLflow version: {mlflow.__version__}")

In [0]:
import mlflow

# Enable autologging with all features
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

## Dataset

The following downloads the [openai/gsm8k](https://huggingface.co/datasets/openai/gsm8k) dataset from Huggingface.

**GSM8K (Grade School Math 8K)** is a dataset of 8.5K high quality linguistically diverse grade school math word problems.
The dataset was created to support the task of question answering on basic mathematical problems that require multi-step reasoning.

This dataset is generally used to test logic and math in language modelling.

---

The code below is borrowed from [Build generative AI apps using DSPy on Databricks](https://docs.databricks.com/aws/en/generative-ai/dspy).

Do I understand what the code does?! Not really 🤷‍♂️ 

In [0]:
%pip install -U pandas fastparquet
dbutils.library.restartPython()

In [0]:
import pandas as pd
print(f"Pandas version: {pd.__version__}")

In [0]:
import pandas as pd
pd.show_versions()

In [0]:
import pandas as pd

def load_data() -> tuple[pd.DataFrame]:
    """
    Load the openai/gsm8k dataset.
    https://huggingface.co/datasets/openai/gsm8k
    """

    # Read train/test split
    file_path = "hf://datasets/openai/gsm8k/main/{}-00000-of-00001.parquet"
    # Workaround for an issue with pyarrow.fs
    # https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html
    train = pd.read_parquet(file_path.format("train"), engine="fastparquet")
    test = pd.read_parquet(file_path.format("test"), engine="fastparquet")

    return train, test

# Create train and test sets containing DSPy
train_dataset, test_dataset = load_data()

print(len(train_dataset), len(test_dataset))

In [0]:
###############################
# This is a copy of dspy.datasets.GSM8K
# Uses pandas.DataFrame instead of Hugging Face's datasets
# There was an issue with the following line:
# dataset = load_dataset("gsm8k", "main")
# it seems that it downloads datasets to a temporary directory
# it failed in Databricks
###############################
import tqdm

hf_official_train = train_dataset
hf_official_test = test_dataset
official_train = []
official_test = []

# adjusted to work with pandas.DataFrame
for _, example in tqdm.tqdm(hf_official_train.iterrows()):
    question = example["question"]

    answer = example["answer"].strip().split()
    assert answer[-2] == "####"

    gold_reasoning = " ".join(answer[:-2])
    answer = str(int(answer[-1].replace(",", "")))

    official_train.append({"question": question, "gold_reasoning": gold_reasoning, "answer": answer})

# adjusted to work with pandas.DataFrame
for _, example in tqdm.tqdm(hf_official_test.iterrows()):
    question = example["question"]

    answer = example["answer"].strip().split()
    assert answer[-2] == "####"

    gold_reasoning = " ".join(answer[:-2])
    answer = str(int(answer[-1].replace(",", "")))

    official_test.append({"question": question, "gold_reasoning": gold_reasoning, "answer": answer})

import random
rng = random.Random(0)
rng.shuffle(official_train)

rng = random.Random(0)
rng.shuffle(official_test)

trainset = official_train[:200]
devset = official_train[200:500]
testset = official_test[:]

import dspy

# known as gsm8k.train in the tutorial
trainset = [dspy.Example(**x).with_inputs("question") for x in trainset]
# gsm8k.dev in the tutorial
devset = [dspy.Example(**x).with_inputs("question") for x in devset]
testset = [dspy.Example(**x).with_inputs("question") for x in testset]

In [0]:
display(train_dataset)

In [0]:
import dspy

lm = dspy.LM(model="databricks/databricks-claude-sonnet-4-5")
dspy.configure(lm=lm)

In [0]:
import mlflow
mlflow.dspy.autolog()

# Define DSPy program

In [0]:
import dspy

qa_system = dspy.ChainOfThought("question -> answer")

In [0]:
# Create and run optimizer with tracking
from dspy.datasets.gsm8k import gsm8k_metric

# MIPROv2 is a DSPy Teleprompter
teleprompter = dspy.teleprompt.MIPROv2(
    metric=gsm8k_metric,
    # one of light, medium, heavy
    # light is the default
    auto="light",
    verbose=True,
)

# The optimization process will automatically be tracked by MLflow DSPy Autologging
optimized_program = teleprompter.compile(
    student=qa_system,
    trainset=trainset,
)


# Review Databricks UI